<a href="https://colab.research.google.com/github/ernestomccormick/NYU-DLSP21/blob/master/SO_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install xmlrpc2 library to connect to Odoo

In [1]:
!pip install xmlrpc2

Import libraries

In [2]:
import xmlrpc.client
from pandas_gbq import to_gbq
import pandas as pd

Authenticate to connect to Biginsights

In [3]:
from google.colab import auth
auth.authenticate_user()

print('Authenticated')

Authenticated


Define GCP project_id

In [4]:
project_id = 'testing-onyx-dwh'


Create Bigquery client

In [5]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)


Connect to Odoo instance

In [6]:
url = 'https://onyxarmor-laboratorio-8153749.dev.odoo.com'
db = 'onyxarmor-laboratorio-8153749'
username = 'plataforma@tacticaweb.com.co'
password = 'ernestonodejadormir'

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
version = common.version()
print("version= ", version)

uid = common.authenticate(db, username, password, {})
print("uid= ", uid)

version=  {'server_version': '14.0+e', 'server_version_info': [14, 0, 0, 'final', 0, 'e'], 'server_serie': '14.0', 'protocol_version': 1}
uid=  2


Define models object

In [7]:
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

# sale.order.line

In [8]:
cols = ['order_id',
        'state',
        'product_id',
        'product_uom_qty',
        'qty_delivered',
        'qty_invoiced',
        'qty_to_invoice',
        'display_name',
        'salesman_id',
        'order_partner_id',
        'invoice_status',
        'price_unit',
        'price_total',
        'price_reduce',
        'company_id']

sales_order_line = models.execute_kw(db, uid, password, 'sale.order.line', 'search_read', [], {'fields': cols})

dfsol = pd.DataFrame.from_dict(sales_order_line)
dfsol.head()

,id,order_id,state,product_id,product_uom_qty,qty_delivered,qty_invoiced,qty_to_invoice,display_name,salesman_id,order_partner_id,invoice_status,price_unit,price_total,price_reduce,company_id
0,31364,"[4320, S04235]",sale,"[8198, PRO - AIR LEVEL II (Gray, XS, -3 inch, ...",10.0,0.0,0.0,0.0,"S04235 - PRO - AIR LEVEL II (Gray, XS, -3 inch...","[63, Javier Rocha]","[189, Premier Body Armor]",no,538.41,5384.10,538.41,"[1, Onyx Protective Group INC]"
1,31363,"[4319, S04234]",sale,"[12271, AETHON RFID (Black, XS, -3 inch, -2 in...",8.0,0.0,0.0,0.0,"S04234 - AETHON RFID (Black, XS, -3 inch, -2 i...","[63, Javier Rocha]","[189, Premier Body Armor]",no,120.45,963.60,120.45,"[1, Onyx Protective Group INC]"
2,31361,"[4318, S04233]",sale,"[12256, 4S17-SAPI-S]",540.0,540.0,540.0,0.0,S04233 - 4S17-SAPI-S,"[62, Javier Luque]","[98, Javier Rocha]",invoiced,236.00,127440.00,236.00,"[1, Onyx Protective Group INC]"
3,31362,"[4318, S04233]",sale,"[12270, Pegasus (Od. Green, XS, -3 inch, -2 in...",1.0,1.0,1.0,0.0,"S04233 - Pegasus (Od. Green, XS, -3 inch, -2 i...","[62, Javier Luque]","[98, Javier Rocha]",invoiced,66.25,66.25,66.25,"[1, Onyx Protective Group INC]"
4,31360,"[4317, S04232]",sale,"[12269, Pegasus (Gray, XS, -3 inch, -2 inch, X...",5.0,0.0,0.0,0.0,"S04232 - Pegasus (Gray, XS, -3 inch, -2 inch, ...","[63, Javier Rocha]","[241, Worldwide Tactical, LLC]",no,66.25,331.25,66.25,"[1, Onyx Protective Group INC]"


In [9]:
dfsol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28322 entries, 0 to 28321
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                28322 non-null  int64  
 1   order_id          28322 non-null  object 
 2   state             28322 non-null  object 
 3   product_id        28322 non-null  object 
 4   product_uom_qty   28322 non-null  float64
 5   qty_delivered     28322 non-null  float64
 6   qty_invoiced      28322 non-null  float64
 7   qty_to_invoice    28322 non-null  float64
 8   display_name      28322 non-null  object 
 9   salesman_id       28322 non-null  object 
 10  order_partner_id  28322 non-null  object 
 11  invoice_status    28322 non-null  object 
 12  price_unit        28322 non-null  float64
 13  price_total       28322 non-null  float64
 14  price_reduce      28322 non-null  float64
 15  company_id        28322 non-null  object 
dtypes: float64(7), int64(1), object(8)
memor

sales.order.line transformation

In [10]:
# Unpacking order_id
dfsol['id_so'] = dfsol['order_id'].str[0]
# Unpacking order_partner_id
dfsol['customer'] = dfsol['order_partner_id'].str[1]
dfsol['customer_id'] = dfsol['order_partner_id'].str[0]
# Unpacking salesman_id
dfsol['salesperson'] = dfsol['salesman_id'].str[1]
dfsol['salesperson_id'] = dfsol['salesman_id'].str[0]
# Unpacking company_id
dfsol['company'] = dfsol['company_id'].str[1]
dfsol['company_id'] = dfsol['company_id'].str[0]
# Unpacking product_id
dfsol['product_id'] = dfsol['product_id'].str[0]
# Changing object types to str or int
dfsol['id_so'] = dfsol['id_so'].astype(int)
dfsol['state'] = dfsol['state'].astype(str)
dfsol['product_id'] = dfsol['product_id'].fillna(0)
dfsol['product_id'] = dfsol['product_id'].astype(int)
dfsol['salesperson_id'] = dfsol['salesperson_id'].fillna(0)
dfsol['salesperson_id'] = dfsol['salesperson_id'].astype(int)
dfsol['salesperson'] = dfsol['salesperson'].astype(str)
dfsol['customer_id'] = dfsol['customer_id'].astype(int)
dfsol['customer'] = dfsol['customer'].astype(str)
dfsol['company_id'] = dfsol['company_id'].astype(int)
dfsol['company'] = dfsol['company'].astype(str)
dfsol['product_id'] = dfsol['product_id'].astype(int)
dfsol['invoice_status'] = dfsol['invoice_status'].astype(str)
dfsol['display_name'] = dfsol['display_name'].astype(str)

# Dropping unnecessry columns
dfsol.drop(['order_partner_id', 'salesman_id', 'order_id'], axis=1, inplace=True)

# Changing column names
dfsol.rename(columns={"id": "id_sol", "company_id": "organization_id_so",
                      "state": "so_status", "company": "organization_name_so", 'opportunity_id':'opportunity_id_so',
                      'product_id':'product_id_so', 'display_name':'product_name_so'}, inplace=True)


dfsol.sort_values('id_sol', ascending=False)

,id_sol,so_status,product_id,product_uom_qty,qty_delivered,qty_invoiced,qty_to_invoice,display_name,invoice_status,price_unit,price_total,price_reduce,organization_id,id_so,customer,customer_id,salesperson,salesperson_id,organization_name
0,31364,sale,8198,10.0,0.0,0.0,0.0,"S04235 - PRO - AIR LEVEL II (Gray, XS, -3 inch...",no,538.41,5384.10,538.41,1,4320,Premier Body Armor,189,Javier Rocha,63,Onyx Protective Group INC
1,31363,sale,12271,8.0,0.0,0.0,0.0,"S04234 - AETHON RFID (Black, XS, -3 inch, -2 i...",no,120.45,963.60,120.45,1,4319,Premier Body Armor,189,Javier Rocha,63,Onyx Protective Group INC
3,31362,sale,12270,1.0,1.0,1.0,0.0,"S04233 - Pegasus (Od. Green, XS, -3 inch, -2 i...",invoiced,66.25,66.25,66.25,1,4318,Javier Rocha,98,Javier Luque,62,Onyx Protective Group INC
2,31361,sale,12256,540.0,540.0,540.0,0.0,S04233 - 4S17-SAPI-S,invoiced,236.00,127440.00,236.00,1,4318,Javier Rocha,98,Javier Luque,62,Onyx Protective Group INC
4,31360,sale,12269,5.0,0.0,0.0,0.0,"S04232 - Pegasus (Gray, XS, -3 inch, -2 inch, ...",no,66.25,331.25,66.25,1,4317,"Worldwide Tactical, LLC",241,Javier Rocha,63,Onyx Protective Group INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28306,590,sale,705,1.0,0.0,0.0,0.0,"S00032 - Namestrip (Yellow, Black) BAIL ENFORC...",no,7.00,7.00,7.00,1,86,The Bunker,245,Tom Bailey,102,Onyx Protective Group INC
28305,589,sale,704,1.0,0.0,0.0,0.0,S00032 - Patrol - Molle exposed zipper Onyx (4...,no,204.00,204.00,204.00,1,86,The Bunker,245,Tom Bailey,102,Onyx Protective Group INC
28304,588,sale,702,1.0,0.0,0.0,0.0,"S00032 - Pegasus (4XL, 4XL, +4 inch, +2 inch, ...",no,0.00,0.00,0.00,1,86,The Bunker,245,Tom Bailey,102,Onyx Protective Group INC
28303,587,sale,701,1.0,0.0,0.0,0.0,"S00032 - PRO - AIR LEVEL II (4XL, 4XL, +4 inch...",no,692.00,692.00,692.00,1,86,The Bunker,245,Tom Bailey,102,Onyx Protective Group INC


In [138]:
dfsol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28322 entries, 0 to 28321
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_sol             28322 non-null  int64  
 1   so_status          28322 non-null  object 
 2   product_id         28322 non-null  int64  
 3   product_uom_qty    28322 non-null  float64
 4   qty_delivered      28322 non-null  float64
 5   qty_invoiced       28322 non-null  float64
 6   qty_to_invoice     28322 non-null  float64
 7   display_name       28322 non-null  object 
 8   invoice_status     28322 non-null  object 
 9   price_unit         28322 non-null  float64
 10  price_total        28322 non-null  float64
 11  price_reduce       28322 non-null  float64
 12  organization_id    28322 non-null  int64  
 13  id_so              28322 non-null  int64  
 14  customer           28322 non-null  object 
 15  customer_id        28322 non-null  int64  
 16  salesperson        283

# sale.order

Load sale.order fields

In [139]:
# models.execute_kw(db, uid, password, 'sale.order', 'fields_get', [], {'attributes': ['string', 'help', 'type']})

Load sale.order data of selected fields

In [140]:
cols = ['name', 'opportunity_id', 'create_date','date_order', 'commitment_date','effective_date', 'partner_shipping_id']

sales_orders = models.execute_kw(db, uid, password, 'sale.order', 'search_read', [], {'fields': cols})

dfso = pd.DataFrame.from_dict(sales_orders)
dfso

,id,name,opportunity_id,create_date,date_order,commitment_date,effective_date,partner_shipping_id
0,4320,S04235,False,2023-10-10 12:36:57,2023-10-10 12:37:00,False,False,"[258, Premier Body Armor, A. Premier - Louis F..."
1,4319,S04234,False,2023-10-10 12:35:55,2023-10-10 12:35:57,False,False,"[258, Premier Body Armor, A. Premier - Louis F..."
2,4318,S04233,False,2023-10-04 19:09:35,2023-10-04 19:11:33,False,2023-10-04,"[98, Javier Rocha]"
3,4317,S04232,False,2023-09-27 16:13:24,2023-09-27 16:13:27,False,False,"[5515, Worldwide Tactical, LLC, AARON DAVIS]"
4,4316,S04231,False,2023-09-27 15:50:32,2023-09-27 15:50:37,False,False,"[258, Premier Body Armor, A. Premier - Louis F..."
...,...,...,...,...,...,...,...,...
4112,86,S00032,False,2021-05-06 01:13:53,2021-05-10 17:08:57,False,2021-06-21,"[245, The Bunker]"
4113,87,S00001,False,2021-05-06 14:50:26,2021-05-10 17:07:00,False,2021-05-25,"[250, PATRICKS UNIFORMS, GEORGIA]"
4114,90,S00004,False,2021-05-06 19:02:30,2021-05-10 17:02:51,False,2021-06-21,"[279, DANA SAFETY SUPPLY, INC, Canton Police D..."
4115,93,S00007,False,2021-05-07 13:10:32,2021-05-10 17:01:28,False,2021-05-26,"[197, EMERGENCY WEAR]"


In [141]:
dfso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   4117 non-null   int64 
 1   name                 4117 non-null   object
 2   opportunity_id       4117 non-null   bool  
 3   create_date          4117 non-null   object
 4   date_order           4117 non-null   object
 5   commitment_date      4117 non-null   object
 6   effective_date       4117 non-null   object
 7   partner_shipping_id  4117 non-null   object
dtypes: bool(1), int64(1), object(6)
memory usage: 229.3+ KB


In [142]:
#dfso[['date_order','effective_date']] = dfso[['date_order','effective_date']].apply(pd.to_datetime)
dfso['effective_date'] = pd.to_datetime(dfso['effective_date'], errors='coerce')
dfso['date_order'] = pd.to_datetime(dfso['date_order'], errors='coerce')
dfso['create_date'] = pd.to_datetime(dfso['create_date'], errors='coerce')
dfso['commitment_date'] = pd.to_datetime(dfso['date_order'], errors='coerce')


dfso['lead_time'] = (dfso['effective_date'] - dfso['date_order']).dt.days
# dfso['customer'] = dfso['partner_id'].str[1]
dfso['customer_shipping_id'] = dfso['partner_shipping_id'].str[0]
# dfso['salesperson'] = dfso['user_id'].str[1]
# dfso['salesperson_id'] = dfso['user_id'].str[0]
# dfso['company'] = dfso['company_id'].str[1]
# dfso['company_id'] = dfso['company_id'].str[0]

dfso.drop(['partner_shipping_id'], axis=1, inplace=True)

dfso.sort_values('lead_time', ascending=False)

dfso.rename(columns={"id": "id_so", "name": "sales_order",}, inplace=True)

# Changing column names
dfso.rename(columns={"create_date_so": "create_date", "date_order": "confirmation_date", "effective_date": "delivery_date", "price_reduce": "discount_unit_price", }, inplace=True)

dfso

,id_so,sales_order,opportunity_id,create_date,date_order,commitment_date,effective_date,lead_time,customer_shipping_id
0,4320,S04235,False,2023-10-10 12:36:57,2023-10-10 12:37:00,2023-10-10 12:37:00,NaT,NaN,258
1,4319,S04234,False,2023-10-10 12:35:55,2023-10-10 12:35:57,2023-10-10 12:35:57,NaT,NaN,258
2,4318,S04233,False,2023-10-04 19:09:35,2023-10-04 19:11:33,2023-10-04 19:11:33,2023-10-04,-1.0,98
3,4317,S04232,False,2023-09-27 16:13:24,2023-09-27 16:13:27,2023-09-27 16:13:27,NaT,NaN,5515
4,4316,S04231,False,2023-09-27 15:50:32,2023-09-27 15:50:37,2023-09-27 15:50:37,NaT,NaN,258
...,...,...,...,...,...,...,...,...,...
4112,86,S00032,False,2021-05-06 01:13:53,2021-05-10 17:08:57,2021-05-10 17:08:57,2021-06-21,41.0,245
4113,87,S00001,False,2021-05-06 14:50:26,2021-05-10 17:07:00,2021-05-10 17:07:00,2021-05-25,14.0,250
4114,90,S00004,False,2021-05-06 19:02:30,2021-05-10 17:02:51,2021-05-10 17:02:51,2021-06-21,41.0,279
4115,93,S00007,False,2021-05-07 13:10:32,2021-05-10 17:01:28,2021-05-10 17:01:28,2021-05-26,15.0,197


In [143]:
dfso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id_so                 4117 non-null   int64         
 1   sales_order           4117 non-null   object        
 2   opportunity_id        4117 non-null   bool          
 3   create_date           4117 non-null   datetime64[ns]
 4   date_order            4117 non-null   datetime64[ns]
 5   commitment_date       4117 non-null   datetime64[ns]
 6   effective_date        3770 non-null   datetime64[ns]
 7   lead_time             3770 non-null   float64       
 8   customer_shipping_id  4117 non-null   int64         
dtypes: bool(1), datetime64[ns](4), float64(1), int64(2), object(1)
memory usage: 261.5+ KB


Merge dfsol con dfso

In [144]:
left_dfsol = dfsol.merge(dfso, on='id_so', how='left')

In [145]:
left_dfsol

,id_sol,so_status,product_id,product_uom_qty,qty_delivered,qty_invoiced,qty_to_invoice,display_name,invoice_status,price_unit,...,salesperson_id,organization_name,sales_order,opportunity_id,create_date,date_order,commitment_date,effective_date,lead_time,customer_shipping_id
0,31364,sale,8198,10.0,0.0,0.0,0.0,"S04235 - PRO - AIR LEVEL II (Gray, XS, -3 inch...",no,538.41,...,63,Onyx Protective Group INC,S04235,False,2023-10-10 12:36:57,2023-10-10 12:37:00,2023-10-10 12:37:00,NaT,NaN,258
1,31363,sale,12271,8.0,0.0,0.0,0.0,"S04234 - AETHON RFID (Black, XS, -3 inch, -2 i...",no,120.45,...,63,Onyx Protective Group INC,S04234,False,2023-10-10 12:35:55,2023-10-10 12:35:57,2023-10-10 12:35:57,NaT,NaN,258
2,31361,sale,12256,540.0,540.0,540.0,0.0,S04233 - 4S17-SAPI-S,invoiced,236.00,...,62,Onyx Protective Group INC,S04233,False,2023-10-04 19:09:35,2023-10-04 19:11:33,2023-10-04 19:11:33,2023-10-04,-1.0,98
3,31362,sale,12270,1.0,1.0,1.0,0.0,"S04233 - Pegasus (Od. Green, XS, -3 inch, -2 i...",invoiced,66.25,...,62,Onyx Protective Group INC,S04233,False,2023-10-04 19:09:35,2023-10-04 19:11:33,2023-10-04 19:11:33,2023-10-04,-1.0,98
4,31360,sale,12269,5.0,0.0,0.0,0.0,"S04232 - Pegasus (Gray, XS, -3 inch, -2 inch, ...",no,66.25,...,63,Onyx Protective Group INC,S04232,False,2023-09-27 16:13:24,2023-09-27 16:13:27,2023-09-27 16:13:27,NaT,NaN,5515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28317,2210,sale,509,0.0,1.0,0.0,1.0,"S00004 - [C018] Pegasus (Black, LG, 0 inch, 0 ...",to invoice,0.00,...,77,Onyx Protective Group INC,S00004,False,2021-05-06 19:02:30,2021-05-10 17:02:51,2021-05-10 17:02:51,2021-06-21,41.0,279
28318,2028,sale,237,1.0,0.0,0.0,0.0,S00004 - Shipping Cost,no,19.00,...,77,Onyx Protective Group INC,S00004,False,2021-05-06 19:02:30,2021-05-10 17:02:51,2021-05-10 17:02:51,2021-06-21,41.0,279
28319,623,sale,734,1.0,1.0,1.0,0.0,"S00007 - PRO - AIR LEVEL II (3XL, 2XL, +4 inch...",invoiced,543.00,...,72,Onyx Protective Group INC,S00007,False,2021-05-07 13:10:32,2021-05-10 17:01:28,2021-05-10 17:01:28,2021-05-26,15.0,197
28320,624,sale,735,1.0,1.0,1.0,0.0,"S00007 - Apollo (3XL, 2XL, +4 inch, +1 inch, 0...",invoiced,0.00,...,72,Onyx Protective Group INC,S00007,False,2021-05-07 13:10:32,2021-05-10 17:01:28,2021-05-10 17:01:28,2021-05-26,15.0,197


In [146]:
left_dfsol.columns

Index(['id_sol', 'so_status', 'product_id', 'product_uom_qty', 'qty_delivered',
       'qty_invoiced', 'qty_to_invoice', 'display_name', 'invoice_status',
       'price_unit', 'price_total', 'price_reduce', 'organization_id', 'id_so',
       'customer', 'customer_id', 'salesperson', 'salesperson_id',
       'organization_name', 'sales_order', 'opportunity_id', 'create_date',
       'date_order', 'commitment_date', 'effective_date', 'lead_time',
       'customer_shipping_id'],
      dtype='object')

# res.partner

In [147]:
# models.execute_kw(db, uid, password, 'res.partner', 'fields_get', [], {'attributes': ['string', 'help', 'type']})

In [148]:
cols = ['name','id', 'parent_name', 'is_company', 'purchase_order_count', 'sale_order_count', 'agent', 'country_id', 'state_id', 'city', 'company_id']

res_partner = models.execute_kw(db, uid, password, 'res.partner', 'search_read', [], {'fields': cols})

dfrp = pd.DataFrame.from_dict(res_partner)
dfrp

,id,name,parent_name,is_company,purchase_order_count,sale_order_count,agent,country_id,state_id,city,company_id
0,7355,Lashanta Gray,False,False,0,0,False,False,False,False,False
1,7357,SCOTT SAVAGE,False,False,0,0,False,False,False,False,False
2,223,10 West Tactical,False,True,0,0,False,"[233, United States]","[52, Texas (US)]",SAN ANTONIO,False
3,224,Ernest Shaun Hernandez,10 West Tactical,False,0,0,False,"[233, United States]","[52, Texas (US)]",SAN ANTONIO,False
4,8284,2XL-Donnell Shuler,False,False,0,0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
8528,1274,wilson,False,False,0,0,False,False,False,False,False
8529,7763,"worldwide tactical,",False,False,0,0,False,False,False,False,False
8530,264,yorhmanarango@hotmail.com,False,False,0,0,False,False,False,False,False
8531,2280,zAXCKERY,False,False,0,0,False,False,False,False,False


In [149]:
dfrp['country'] = dfrp['country_id'].str[1]
dfrp['country_id'] = dfrp['country_id'].str[0]
dfrp['state'] = dfrp['state_id'].str[1]
dfrp['state_id'] = dfrp['state_id'].str[0]
dfrp['company'] = dfrp['company_id'].str[1]
dfrp['company_id'] = dfrp['company_id'].str[0]

dfrp["country_id"] = dfrp["country_id"].fillna(0)
dfrp["state_id"] = dfrp["state_id"].fillna(0)

dfrp.loc[dfrp['parent_name'] == False,['parent']] = dfrp['name']
dfrp.loc[dfrp['parent_name'] != False,['parent']] = dfrp['parent_name']

dfrp['state'] = dfrp['state'].str.split(' \(', expand=True)[0]

dfrp = dfrp.astype({"country_id":'int', "state_id":'int'})
dfrp = dfrp.astype({"name":'str', "parent_name":'str', "city":'str', "country":'str', "state":'str'})

dfrp

,id,name,parent_name,is_company,purchase_order_count,sale_order_count,agent,country_id,state_id,city,company_id,country,state,company,parent
0,7355,Lashanta Gray,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,Lashanta Gray
1,7357,SCOTT SAVAGE,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,SCOTT SAVAGE
2,223,10 West Tactical,False,True,0,0,False,233,52,SAN ANTONIO,NaN,United States,Texas,NaN,10 West Tactical
3,224,Ernest Shaun Hernandez,10 West Tactical,False,0,0,False,233,52,SAN ANTONIO,NaN,United States,Texas,NaN,10 West Tactical
4,8284,2XL-Donnell Shuler,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,2XL-Donnell Shuler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,1274,wilson,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,wilson
8529,7763,"worldwide tactical,",False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,"worldwide tactical,"
8530,264,yorhmanarango@hotmail.com,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,yorhmanarango@hotmail.com
8531,2280,zAXCKERY,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,zAXCKERY


In [150]:
# Find unique values of a column
print(dfrp['company'].unique())
print(left_dfsol['organization_name'].unique())


[nan 'Onyx Protective Group INC']
['Onyx Protective Group INC' 'Usewing Corp']


In [151]:
dfrp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8533 entries, 0 to 8532
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8533 non-null   int64  
 1   name                  8533 non-null   object 
 2   parent_name           8533 non-null   object 
 3   is_company            8533 non-null   bool   
 4   purchase_order_count  8533 non-null   int64  
 5   sale_order_count      8533 non-null   int64  
 6   agent                 8533 non-null   bool   
 7   country_id            8533 non-null   int64  
 8   state_id              8533 non-null   int64  
 9   city                  8533 non-null   object 
 10  company_id            1 non-null      float64
 11  country               8533 non-null   object 
 12  state                 8533 non-null   object 
 13  company               1 non-null      object 
 14  parent                8533 non-null   object 
dtypes: bool(2), float64(1

In [152]:
dfrp.columns

Index(['id', 'name', 'parent_name', 'is_company', 'purchase_order_count',
       'sale_order_count', 'agent', 'country_id', 'state_id', 'city',
       'company_id', 'country', 'state', 'company', 'parent'],
      dtype='object')

In [153]:
# Dropping unnecessary columns
dfso.drop(['country_id', 'state_id', ], axis=1, inplace=True)

dfrp.rename(columns={"id": "id_rp", "name": "entity_name", "company_id_rp": "organization_id_rp", "company": "organization_name_rp"}, inplace=True)
dfrp

,id_rp,entity_name,parent_name,is_company,purchase_order_count,sale_order_count,agent,country_id,state_id,city,company_id,country,state,company,parent
0,7355,Lashanta Gray,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,Lashanta Gray
1,7357,SCOTT SAVAGE,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,SCOTT SAVAGE
2,223,10 West Tactical,False,True,0,0,False,233,52,SAN ANTONIO,NaN,United States,Texas,NaN,10 West Tactical
3,224,Ernest Shaun Hernandez,10 West Tactical,False,0,0,False,233,52,SAN ANTONIO,NaN,United States,Texas,NaN,10 West Tactical
4,8284,2XL-Donnell Shuler,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,2XL-Donnell Shuler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,1274,wilson,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,wilson
8529,7763,"worldwide tactical,",False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,"worldwide tactical,"
8530,264,yorhmanarango@hotmail.com,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,yorhmanarango@hotmail.com
8531,2280,zAXCKERY,False,False,0,0,False,0,0,False,NaN,nan,nan,NaN,zAXCKERY


# product.product

In [22]:
cols = ['id',
        'display_name',
        'product_tmpl_id',
        'categ_id',
        'price',
        'sale_ok',
        'purchase_ok',
        'uom_name',
        'is_product_variant',
        'has_configurable_attributes',
        'attribute_line_ids',
        'sale_delay',
        'reordering_min_qty',
        'reordering_max_qty',
        'produce_delay',
        'create_date']

# product = models.execute_kw(db, uid, password, 'product.template', 'fields_get', [], {'fields': cols})

product = models.execute_kw(db, uid, password, 'product.product', 'search_read', [], {'fields': cols})


dfp = pd.DataFrame.from_dict(product)
dfp

,id,display_name,product_tmpl_id,categ_id,price,sale_ok,purchase_ok,uom_name,is_product_variant,has_configurable_attributes,attribute_line_ids,sale_delay,reordering_min_qty,reordering_max_qty,produce_delay,create_date
0,6833,"[1P3L502XL12P] PHOENIX (Gray, 3XL, +5 inch, 0 ...","[25, PHOENIX]","[7, All / Saleable / Panel]",0.0,True,False,Units,True,True,"[56, 192, 193, 194, 195, 196, 242]",30.0,0.0,0.0,25.0,2022-07-28 14:52:01
1,7778,"[1P3XL302XL1] PHOENIX (Gray, 3XL, +3 inch, 0 i...","[25, PHOENIX]","[7, All / Saleable / Panel]",0.0,True,False,Units,True,True,"[56, 192, 193, 194, 195, 196, 242]",30.0,0.0,0.0,25.0,2022-10-31 21:03:36
2,7777,"[1P3XL302XL1] Pegasus (Black, 3XL, +3 inch, 0 ...","[35, Pegasus]","[4, All / Saleable / Carrier]",0.0,True,False,Units,True,True,"[66, 274, 275, 276, 277, 278]",30.0,0.0,0.0,25.0,2022-10-31 21:00:29
3,7776,"[1PLG10L0] PHOENIX (Gray, LG, -1 inch, 0 inch,...","[25, PHOENIX]","[7, All / Saleable / Panel]",0.0,True,False,Units,True,True,"[56, 192, 193, 194, 195, 196, 242]",30.0,0.0,0.0,25.0,2022-10-31 20:52:53
4,7036,"[1PMD30SM2] Pegasus (Black, MD, +3 inch, 0 inc...","[35, Pegasus]","[4, All / Saleable / Carrier]",0.0,True,False,Units,True,True,"[66, 274, 275, 276, 277, 278]",30.0,0.0,0.0,25.0,2022-08-23 18:52:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11129,2999,"Zipper Separable Molded #5 20"" (White)","[414, Zipper Separable Molded #5 20""]","[11, All / Expenses / Materials / No Ballistic]",0.0,True,True,Units,True,True,[874],0.0,0.0,0.0,0.0,2021-09-23 14:55:42
11130,6444,[P020] KOOZIE BALLISTIC / 611,"[686, [P020] KOOZIE BALLISTIC / 611]","[7, All / Saleable / Panel]",0.0,True,False,Units,True,False,[],30.0,0.0,0.0,25.0,2022-06-29 16:22:13
11131,3878,fees,"[507, fees]","[1, All]",0.0,True,True,Units,True,False,[],0.0,0.0,0.0,0.0,2021-11-17 16:22:04
11132,11963,patro;,"[948, patro;]","[1, All]",0.0,True,True,Units,True,False,[],0.0,0.0,0.0,0.0,2023-09-11 16:23:46


In [23]:
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11134 entries, 0 to 11133
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           11134 non-null  int64  
 1   display_name                 11134 non-null  object 
 2   product_tmpl_id              11134 non-null  object 
 3   categ_id                     11134 non-null  object 
 4   price                        11134 non-null  float64
 5   sale_ok                      11134 non-null  bool   
 6   purchase_ok                  11134 non-null  bool   
 7   uom_name                     11134 non-null  object 
 8   is_product_variant           11134 non-null  bool   
 9   has_configurable_attributes  11134 non-null  bool   
 10  attribute_line_ids           11134 non-null  object 
 11  sale_delay                   11134 non-null  float64
 12  reordering_min_qty           11134 non-null  float64
 13  reordering_max_q

In [24]:
# Changing format date
dfp['create_date'] = pd.to_datetime(dfp['create_date'], errors='coerce')
# Unpacking display_name
dfp['full_product_name'] = dfp['display_name'].astype(str)
# Unpacking product_tmpl_id
dfp['product_template'] = dfp['product_tmpl_id'].str[1]
dfp['product_tmpl_id'] = dfp['product_tmpl_id'].str[0]
# Unpacking categ_id
dfp['product_category_name'] = dfp['categ_id'].str[1]
dfp['categ_id'] = dfp['categ_id'].str[0]
# Changing object types to str or int
dfp['full_product_name'] = dfp['full_product_name'].astype(str)

dfp['uom_name'] = dfp['uom_name'].astype(str)

# Dropping unnecessry columns
dfp.drop(['product_tmpl_id', 'categ_id', 'display_name'], axis=1, inplace=True)

# Changing column names
dfp.rename(columns={"id": "id_p"}, inplace=True)


dfp.sort_values('id_p', ascending=False)

,id_p,price,sale_ok,purchase_ok,uom_name,is_product_variant,has_configurable_attributes,attribute_line_ids,sale_delay,reordering_min_qty,reordering_max_qty,produce_delay,create_date,full_product_name,product_template,product_category_name
201,12271,0.0,True,False,Units,True,True,"[1601, 1602, 1603, 1604, 1605, 1606]",30.0,0.0,0.0,25.0,2023-10-10 12:35:50,"AETHON RFID (Black, XS, -3 inch, -2 inch, XS, ...",AETHON RFID,All / Saleable / Carrier
9957,12270,0.0,True,False,Units,True,True,"[66, 274, 275, 276, 277, 278]",30.0,0.0,0.0,25.0,2023-10-04 19:16:34,"Pegasus (Od. Green, XS, -3 inch, -2 inch, XS, ...",Pegasus,All / Saleable / Carrier
9956,12269,0.0,True,False,Units,True,True,"[66, 274, 275, 276, 277, 278]",30.0,0.0,0.0,25.0,2023-09-27 16:13:06,"Pegasus (Gray, XS, -3 inch, -2 inch, XS, -3 inch)",Pegasus,All / Saleable / Carrier
5856,12268,0.0,True,False,Units,True,True,"[60, 325, 326, 327, 328, 329, 330]",30.0,0.0,0.0,25.0,2023-09-26 15:59:42,"PRO-AIR LEVEL II STRUCTURED FEMALE (Gray, XL, ...",PRO-AIR LEVEL II STRUCTURED FEMALE,All / Saleable / Panel
6518,12267,0.0,True,False,Units,True,True,"[280, 356, 357, 358, 359, 360, 361, 362]",30.0,0.0,0.0,25.0,2023-09-26 15:13:30,"PRO-AIR LEVEL IIIA STRUCTURED FEMALE (Gray, XS...",PRO-AIR LEVEL IIIA STRUCTURED FEMALE,All / Saleable / Panel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,68,0.0,True,True,Yd width 60 inch,True,True,[44],0.0,40.0,200.0,0.0,2021-03-25 15:34:41,"Foam 1/4""","Foam 1/4""",All / Expenses / Materials / No Ballistic
1846,66,0.0,False,True,Yd width 63 inch,True,False,[],0.0,200.0,0.0,0.0,2021-03-25 15:29:49,DSM 117,DSM 117,All / Expenses / Materials / Ballistic
1845,65,0.0,False,True,Yd width 63 inch,True,False,[],0.0,200.0,0.0,0.0,2021-03-25 15:28:24,DSM 115,DSM 115,All / Expenses / Materials / Ballistic
1686,56,0.0,True,True,Units,True,True,[24],0.0,0.0,0.0,0.0,2021-03-25 15:17:13,"Buckle Female/Male 1"" (Black)","Buckle Female/Male 1""",All / Expenses / Materials / No Ballistic


# product.attribute

In [ ]:
cols = ['id','name','value_ids']

product_attributes = models.execute_kw(db, uid, password, 'product.attribute', 'search_read', [], {'fields': cols})

dfpa = pd.DataFrame.from_dict(product_attributes)
dfpa

In [154]:
left_dfsol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28322 entries, 0 to 28321
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id_sol                28322 non-null  int64         
 1   so_status             28322 non-null  object        
 2   product_id            28322 non-null  int64         
 3   product_uom_qty       28322 non-null  float64       
 4   qty_delivered         28322 non-null  float64       
 5   qty_invoiced          28322 non-null  float64       
 6   qty_to_invoice        28322 non-null  float64       
 7   display_name          28322 non-null  object        
 8   invoice_status        28322 non-null  object        
 9   price_unit            28322 non-null  float64       
 10  price_total           28322 non-null  float64       
 11  price_reduce          28322 non-null  float64       
 12  organization_id       28322 non-null  int64         
 13  id_so           

In [ ]:
from pandas_gbq import to_gbq

In [ ]:
# Define your BigQuery dataset and table
dataset_name = 'operational'
table_name = 'sales_orders_lines'

# Full table reference
full_table_name = f"{dataset_name}.{table_name}"

# Upload the DataFrame to BigQuery
to_gbq(left_dfsol, full_table_name, project_id=project_id, if_exists='replace')  # use 'append' if you want to add to an existing table instead of replacing it

In [ ]:
# Define your BigQuery dataset and table
dataset_name = 'operational'
table_name = 'res_partner'

# Full table reference
full_table_name = f"{dataset_name}.{table_name}"

to_gbq(dfrp, full_table_name, project_id=project_id, if_exists='replace')  # use 'append' if you want to add to an existing table instead of replacing it

# Common errors


ArrowTypeError: Expected bytes, got a 'bool' object >>>> Change datatype object